In [3]:
import json, os, sys, random
import tensorflow as tf
import numpy as np
import random 
import pandas as pd
from sklearn.model_selection import train_test_split

In [4]:
infile = open("./poker.jsonl","r")
data = []

for line in infile:
    data.append(json.loads(line))
print(random.choice(data))

print(len(data))
print(pd.DataFrame(data))

# print(data[:, 2:5])


['12:01:32.431 PM 04-Mar-2024', 'BET', {'name': 'charlie', 'bet': 3.0, 'folded': False, 'round_winner_by_cards': 'danny', 'chips': 13.0, 'face_down_cards': [['7', 'heart'], ['4', 'club'], ['4', 'diamond']], 'face_up_cards': [['3', 'club']], 'face_player_cards': [['9', 'diamond']], 'all_cards': [['4', 'club'], ['4', 'diamond'], ['9', 'diamond'], ['7', 'heart'], ['3', 'club']], 'handvalue': 'one pair'}, {'name': 'danny', 'bet': 3.0, 'folded': False, 'round_winner_by_cards': 'danny', 'chips': 42.0, 'face_down_cards': [['king', 'club'], ['jack', 'club'], ['king', 'diamond']], 'face_up_cards': [['2', 'heart']], 'face_player_cards': [['ace', 'club']], 'all_cards': [['king', 'diamond'], ['2', 'heart'], ['ace', 'club'], ['jack', 'club'], ['king', 'club']], 'handvalue': 'one pair'}, {'name': 'elissa', 'bet': 0.0, 'folded': True, 'round_winner_by_cards': 'danny', 'chips': 0.0, 'face_down_cards': [], 'face_up_cards': [], 'face_player_cards': [['2', 'club'], ['6', 'diamond'], ['3', 'spade'], ['9',

In [53]:
def card_to_feature(cardArray, size, featureMap, featureName):
    featureMap[f'{featureName}_num'] = len(cardArray)
    for n in range(size):
        if n < len(cardArray):
            suit = 0
            match cardArray[n][1]:
                case 'club':
                    suit = 0
                case 'diamond':
                    suit = 1
                case 'heart':
                    suit = 2
                case 'spade':
                    suit = 3
            featureMap[f'{featureName}_{n}_suit'] = suit

            match cardArray[n][0]:
                case 'jack':
                    number = 11
                case 'queen':
                    number = 12
                case 'king':
                    number = 13
                case 'ace':
                    number = 1
                case _:
                    number = int(cardArray[n][0])
            featureMap[f'{featureName}_{n}_number'] = number
        else:
            
            featureMap[f'{featureName}_{n}_suit'] = 0
            featureMap[f'{featureName}_{n}_number'] = 0

def person_to_feature(personMap, nextPersonMap, people):
    conditionInput = {}
    # print(actor, "knows", person['bet'], person['folded'], person['face_up_cards'])
    conditionInput['self_bet'] = personMap['bet']
    conditionInput['self_folded'] = int(personMap['folded'])
    
    faceUpCards = personMap['face_up_cards']
    featureName = 'self_face_up_card'
    card_to_feature(faceUpCards, 4, conditionInput, featureName)

    faceUpCards = personMap['face_player_cards']
    featureName = 'self_face_card'
    card_to_feature(faceUpCards, 5, conditionInput, featureName)

    
    otherNum = 0
    for k in range(len(people)):
        if j != k:
            otherNum+=1
            other = people[k]
            # print(other['name'], other['bet'], other['folded'], other['face_up_cards'])
            conditionInput[f'other_{otherNum}_bet'] = other['bet']
            conditionInput[f'other_{otherNum}_folded'] = int(other['folded'])
            
            faceUpCards = other['face_up_cards']
            featureName = f'other_{otherNum}_face_up_card'
            card_to_feature(faceUpCards, 4, conditionInput, featureName)
    # print("THIS INPUT", conditionInput)

    output = {}
    output['bet'] = nextPersonMap['bet'] - personMap['bet']
    output['folded'] = int(nextPersonMap['folded'])

    return conditionInput, output

In [55]:
features = []
labels = []
end = 0
current = 0
while current < len(data): 
    
    #find the range of the round
    start = -1  
    current = end + 1
    while(current < len(data)-1 and data[current][1] != "WIN"):
        # print(data[current][1])
        if start < 0 and data[current][1] == 'BET':
            # print("start at ", current)
            start = current
        
        current+=1

    
    # print("end at ", current)
    end = current

    if(current < len(data)-1):
        print('from ', start, 'to ', end)
        winner = data[end][2]['round_winner_by_cards']
        print(winner, 'is the winner')
        
        betNum = 0
        noAction = True
        for i in range(start, end):
            #compare this round and next round
            #check next round change:
            round = data[i]
            nextRound = data[i+1]

            actor = 0
            
            if(round[1] == "BET"):
                betNum+=1

                # if i < 75 and i > 41:
                #     print("round: ", i, "betNum: ", betNum)    
                # for person in round[2:5]:
                #     print(person)
                # print(round)
                people = round[2:5]
                for j in range(len(people)):
                    person = people[j]
                    name = person['name']
                    bet = person['bet']
                    folded = person['folded']
                    # faceDownCardsNum = len(person['face_down_cards'])
                    
                    if (name == winner):
                        for nextPerson in nextRound[2:5]:
                            if name == nextPerson['name']:
                                #check change
                                nextBet = nextPerson['bet']
                                nextFolded = nextPerson['folded']
                                # nextFaceDownCardsNum = len(nextPerson['face_down_cards'])

                                if (bet != nextBet) or (folded != nextFolded):
                                    #Action taken
                                    actor = name
                                    noAction = False
                                    #print action condition:
                                    conditionInput, output = person_to_feature(person, nextPerson, people)
                                    features.append(conditionInput)
                                    labels.append(output)
                                    # print("ACTION\t\t", "THIS OUTPUT", output, " THIS INPUT", conditionInput)

                                    if (betNum == 3):
                                        betNum = 0
                                        noAction = True
                                elif(betNum == 3):
                                    betNum = 0
                                    if(noAction):
                                        #Action taken
                                        actor = name

                                        #print action condition:
                                        conditionInput, output = person_to_feature(person, nextPerson, people)
                                        features.append(conditionInput)
                                        labels.append(output)
                                        # print("NO ACTION\t", "THIS OUTPUT", output, " THIS INPUT", conditionInput)
                                    else:
                                        noAction = True
            elif(round[1] == "WIN"):
                print("WIN")
            elif(round[1] == "ENDROUND"):
                # print("ENDROUND")
                pass
# print("sampleWithLabel:", features)    
    
X = pd.DataFrame(features)
y = pd.DataFrame(labels)
# print(df[0][1])

print(X)
print(y)

from  3 to  37
danny is the winner
from  41 to  75
bob is the winner
from  79 to  112
bob is the winner
from  116 to  141
bob is the winner
from  145 to  168
bob is the winner
from  172 to  206
bob is the winner
from  210 to  241
danny is the winner
from  245 to  283
bob is the winner
from  287 to  313
danny is the winner
from  317 to  342
bob is the winner
from  346 to  372
danny is the winner
from  376 to  400
danny is the winner
from  404 to  430
bob is the winner
from  434 to  458
bob is the winner
from  462 to  486
danny is the winner
from  490 to  515
bob is the winner
from  519 to  544
bob is the winner
from  548 to  572
danny is the winner
from  576 to  602
bob is the winner
from  606 to  633
bob is the winner
from  637 to  660
bob is the winner
from  664 to  698
bob is the winner
from  702 to  739
bob is the winner
from  743 to  754
bob is the winner
from  758 to  795
charlie is the winner
from  799 to  836
charlie is the winner
from  840 to  870
charlie is the winner
from  87

In [56]:
x_train, x_test, y_train, y_test = train_test_split(X.to_numpy(), y.to_numpy(), test_size=0.2, random_state=42)
display(x_train.shape)
display(y_train.shape)

(265, 44)

(265, 2)

In [57]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64),
    # tf.keras.layers.Dense(128),
    # tf.keras.layers.Dense(256),
    # tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(256, activation='relu'),
    # tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(2)
])
loss_fn = tf.keras.losses.MeanAbsoluteError() #BinaryFocalCrossentropy(from_logits=True)
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=tf.keras.metrics.MeanSquaredError())

In [58]:
model.fit(x_train, y_train, epochs=100)

Epoch 1/100
9/9 [==============================] - 1s 2ms/step - loss: 0.8364 - mean_squared_error: 1.3984
Epoch 2/100
9/9 [==============================] - 0s 2ms/step - loss: 0.6118 - mean_squared_error: 0.8867
Epoch 3/100
9/9 [==============================] - 0s 1ms/step - loss: 0.5208 - mean_squared_error: 0.6722
Epoch 4/100
9/9 [==============================] - 0s 1ms/step - loss: 0.4632 - mean_squared_error: 0.5574
Epoch 5/100
9/9 [==============================] - 0s 1ms/step - loss: 0.4148 - mean_squared_error: 0.4844
Epoch 6/100
9/9 [==============================] - 0s 1ms/step - loss: 0.3687 - mean_squared_error: 0.4566
Epoch 7/100
9/9 [==============================] - 0s 1ms/step - loss: 0.3446 - mean_squared_error: 0.3979
Epoch 8/100
9/9 [==============================] - 0s 1ms/step - loss: 0.3506 - mean_squared_error: 0.3740
Epoch 9/100
9/9 [==============================] - 0s 1ms/step - loss: 0.3363 - mean_squared_error: 0.3440
Epoch 10/100
9/9 [===================

In [63]:
model.evaluate(x_test,  y_test, verbose=2)
model.save("poker.keras")

3/3 - 0s - loss: 0.3624 - mean_squared_error: 0.4470 - 22ms/epoch - 7ms/step


In [64]:
start = random.randint(0,len(x_train))
end = start + 10
print(start)
predictions = model(x_train[start:end]).numpy()
display(list(np.round(tf.nn.softmax(predictions).numpy(),3)[:,1]))
display(list(map(lambda xy: xy[1], y_train[start:end])))

147


[0.498, 0.446, 0.133, 0.512, 0.289, 0.241, 0.475, 0.244, 0.262, 0.282]

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [65]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 64)                2880      
                                                                 
 dense_7 (Dense)             (None, 256)               16640     
                                                                 
 dense_8 (Dense)             (None, 2)                 514       
                                                                 
Total params: 20,034
Trainable params: 20,034
Non-trainable params: 0
_________________________________________________________________
